In [1]:
from OpenSSL import crypto
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding as sym_padding
import os

In [2]:
# START OF CREATING PUBLIC-PRIVATE KEY PAIR
# Generate a key pair of type RSA and size 2048 bits
key = crypto.PKey()
key.generate_key(crypto.TYPE_RSA, 2048)

In [3]:
private_key = crypto.dump_privatekey(crypto.FILETYPE_PEM, key).decode('utf-8')

In [4]:
public_key = crypto.dump_publickey(crypto.FILETYPE_PEM, key).decode('utf-8')

In [5]:
# print("Private Key:")
# print(private_key)

# print("Public Key:")
# print(public_key)

# Optionally, you can write these keys to files
with open("CS447 Project/Key-Pairs/private_key.pem", "w") as f:
    f.write(private_key)

with open("CS447 Project/Key-Pairs/public_key.pem", "w") as f:
    f.write(public_key)

# END OF CREATING PUBLIC-PRIVATE KEY PAIR

In [6]:
#START OF ENCRYPTING THE FILE

# Load the public key
with open("CS447 Project/Key-Pairs/public_key.pem", "rb") as key_file:
    public_key = serialization.load_pem_public_key(
        key_file.read(),
        backend=default_backend()
    )

In [7]:
# Generate a random symmetric key (for AES encryption)
symmetric_key = os.urandom(32)  # AES-256 key

In [8]:
# Encrypt the symmetric key using the public key
encrypted_symmetric_key = public_key.encrypt(
    symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

In [9]:
# Read the file to encrypt
file_path = 'CS447 Project/Key-Pairs/test.txt'
with open(file_path, 'rb') as f:
    file_data = f.read()

In [10]:
# Pad and encrypt the file data using the symmetric key
padder = sym_padding.PKCS7(128).padder() 
padded_data = padder.update(file_data) + padder.finalize()

In [11]:
cipher = Cipher(algorithms.AES(symmetric_key), modes.CBC(os.urandom(16)), backend=default_backend())
encryptor = cipher.encryptor()
encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

In [12]:
# Save the encrypted data and the encrypted symmetric key
with open('CS447 Project/Key-Pairs/encrypted_file', 'wb') as f:
    f.write(encrypted_data)

In [13]:
with open('CS447 Project/Key-Pairs/encrypted_key', 'wb') as f:
    f.write(encrypted_symmetric_key)

#END OF ENCRYPTING THE FILE

In [14]:
#START OF DECRYPTING THE FILE
# Load the private key
with open("CS447 Project/Key-Pairs/private_key.pem", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )

In [15]:
# Load the encrypted symmetric key
with open('CS447 Project/Key-Pairs/encrypted_key', 'rb') as f:
    encrypted_symmetric_key = f.read()

# Decrypt the symmetric key
symmetric_key = private_key.decrypt(
    encrypted_symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

In [16]:
# Load the encrypted file data
with open('CS447 Project/Key-Pairs/encrypted_file', 'rb') as f:
    iv = f.read(16)  # Assuming a 16-byte IV for AES
    encrypted_data = f.read()

In [17]:
# Decrypt the file data using the symmetric key and the IV
cipher = Cipher(algorithms.AES(symmetric_key), modes.CBC(iv), backend=default_backend())
decryptor = cipher.decryptor()
padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

In [18]:
# Unpad the decrypted data
unpadder = sym_padding.PKCS7(128).unpadder()
decrypted_data = unpadder.update(padded_data) + unpadder.finalize()

In [19]:
# Save or process the decrypted data
with open('CS447 Project/Key-Pairs/decrypted_file', 'wb') as f:
    f.write(decrypted_data)